In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
data = [train['text'],test['text']]

In [ ]:
for text in data:
    

    text=text.lower()
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    #Replace &amp, &lt, &gt with &,<,> respectively
    text=text.replace(r'&amp;?',r'and')
    text=text.replace(r'&lt;',r'<')
    text=text.replace(r'&gt;',r'>')
    #remove hashtag sign
    #text=re.sub(r"#","",text)   
    #remove mentions
    text = re.sub(r"(?:\@)\w+", '', text)
    #text=re.sub(r"@","",text)
    #remove non ascii chars
    text=text.encode("ascii",errors="ignore").decode()
    #remove some puncts (except . ! ?)
    text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
    text=re.sub(r'[!]+','!',text)
    text=re.sub(r'[?]+','?',text)
    text=re.sub(r'[.]+','.',text)
    text=re.sub(r"'","",text)
    text=re.sub(r"\(","",text)
    text=re.sub(r"\)","",text)
    
    text=" ".join(text.split())


train=train[train["text"]!='']

In [ ]:
corpus1 = []
for i in range(0, 7613):
  text = re.sub('[^a-zA-Z]', ' ', train['text'][i])
  text = text.lower()
  text = text.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus1.append(text)

In [ ]:
print(corpus1)

In [ ]:
corpus2 = []
for i in range(0, 3263):
  text = re.sub('[^a-zA-Z]', ' ', test['text'][i])
  text = text.lower()
  text = text.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus2.append(text)

In [ ]:
print(corpus2)

In [ ]:
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus1).toarray()
y = train.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
train_accuracies = {'Logistic Regression':0, 'Random Forest Classifier':0}
test_accuracies = {'Logistic Regression':0,   'Random Forest Classifier':0}

In [ ]:
lgr = LogisticRegression()
lgr.fit(X_train, y_train)

In [ ]:
train_preds = lgr.predict(X_train)
test_preds = lgr.predict(X_test)
print("Logistic Regression results :")
print("   -   F1 Score on the train set : {:.2f}%".format(f1_score(train_preds, y_train)*100))
train_accuracies['Logistic Regression'] = f1_score(train_preds, y_train)*100
print("   -   F1 Score on the test set : {:.2f}%".format(f1_score(test_preds, y_test)*100))
test_accuracies['Logistic Regression'] = f1_score(test_preds, y_test)*100

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)
print("Random Forest Classifier results :")
print("   -   F1 Score on the train set : {:.2f}%".format(f1_score(train_preds, y_train)*100))
train_accuracies['Random Forest Classifier'] = f1_score(train_preds, y_train)*100
print("   -   F1 Score on the test set : {:.2f}%".format(f1_score(test_preds, y_test)*100))
test_accuracies['Random Forest Classifier'] = f1_score(test_preds, y_test)*100

In [ ]:
ind = np.arange(2)
width = 0.27

fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)

rects1 = ax.bar(ind, list(train_accuracies.values()), width, color='b')
rects2 = ax.bar(ind+width, list(test_accuracies.values()), width, color='g')

ax.set_ylabel('F1 Score (%)')
ax.set_xticks(ind+width)
ax.set_xticklabels( ('Logistic Regression', 'Random Forest Classifier') )
ax.legend((rects1[0], rects2[0]), ('Train Score', 'Test Score'))

plt.ylim((0,100))
plt.show()

In [ ]:
test

In [ ]:
test2 = cv.fit_transform(corpus2).toarray()
predictions = rf.predict(test2)

In [ ]:
output = pd.DataFrame({'id': test.id ,'target': predictions})

filename = 'My_Predictions.csv'

output.to_csv(filename,index=False)

print('Saved file: ' + filename)